In questo notebook vedremo come utilizzare le librerie KerasTuner ed AutoKeras per migliorare le performance della nostra rete neurale. Agiremo sul dataset **diabetes**, affrontando un problema di regressione, e cercando le combinazioni di iperparametri ed architetture migliori.

Per prima cosa, importiamo i package necessari.

In [32]:
import autokeras as ak
import keras_tuner as kt
from tensorflow import keras

from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

**Parte 1: tuning degli iperparametri**

Creiamo una funzione che accetta un parametro `hp` rappresentativo di un container contenente una rappresentazione dello spazio degli iperparametri. All'interno della funzione creeremo un modello di tipo sequenziale, nel quale andremo a definire una possibile scelta di iperparametri.

In [33]:
def build_model(hp: kt.HyperParameters) -> keras.models.Sequential:
    """Builds a neural network model with tunable hyperparameters.

    Args:
        hp (HyperParameters): An instance of HyperParameters from KerasTuner.

    Returns:
        keras.models.Sequential: The compiled model.

    """
    model = keras.Sequential()
    model.add(keras.layers.Dense(
        units=hp.Int('units', min_value=8, max_value=64, step=8),
        activation='relu'))
    model.add(keras.layers.Dense(
        units=hp.Int('units', min_value=8, max_value=128, step=8),
        activation='relu'))
    if hp.Boolean('dropout'):
        model.add(keras.layers.Dropout(rate=0.2))
    model.add(keras.layers.Dense(
        1,
        activation='relu'))
    model.compile(loss=keras.losses.MeanAbsoluteError(),
        optimizer=keras.optimizers.Adam())
    return model

Creiamo adesso un oggetto di tipo `RandomSearch` che accetta:

* il metodo `build_model`, necessario a creare il modello;
* il parametro `objective`, che indica qual è il valore che vogliamo ottimizzare;
* il parametro `max_trials`, che indica qual è il numero massimo di tentativi che vogliamo effettuare.

In [34]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    directory='tuning',
    project_name='pcs-hyp',
    max_trials=10,
    seed=42)

Carichiamo il dataset, suddividiamolo in insiemi di training e test, ed usiamo il metodo `search` per addestrare il nostro modello. L'interfaccia di questo metodo è molto simile a quella del metodo `fit` di Keras.

In [35]:
X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y)

tuner.search(
    X_train,
    y_train,
    epochs=10,
    validation_data=(X_test, y_test))

Trial 10 Complete [00h 00m 01s]
val_loss: 155.11521911621094

Best val_loss So Far: 141.21014404296875
Total elapsed time: 00h 00m 10s
INFO:tensorflow:Oracle triggered exit


Vediamo quali sono gli iperparametri del miglior modello ottenuto.

In [36]:
best_model = tuner.get_best_models()[0]
best_model.fit(X_train, y_train)
best_model.summary()

11/11 [==============================] - 0s 800us/step - loss: 129.5499
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                704       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 4,929
Trainable params: 4,929
Non-trainable params: 0
_________________________________________________________________


**Parte 2: AutoML**

Sfruttiamo l'oggetto `StructuredDataRegressor` di AutoKeras per determinare in maniera automatica la migliore struttura possibile per la regressione.

In [37]:
reg = ak.StructuredDataRegressor(
    loss=keras.losses.MeanAbsoluteError(),
    metrics=[keras.metrics.MeanAbsoluteError()],
    directory='tuning',
    project_name='pcs-ak',
    max_trials=10)

reg.fit(
    X_train,
    y_train,
    epochs=10,
    validation_data=(X_test, y_test))

Trial 10 Complete [00h 00m 02s]
val_loss: 56.51576614379883

Best val_loss So Far: 56.51576614379883
Total elapsed time: 00h 00m 16s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
11/11 [==============================] - 0s 12ms/step - loss: 149.6962 - mean_absolute_error: 126.2956 - val_loss: 158.3270 - val_mean_absolute_error: 158.3270
Epoch 2/10
11/11 [==============================] - 0s 4ms/step - loss: 148.8982 - mean_absolute_error: 148.8982 - val_loss: 157.1462 - val_mean_absolute_error: 157.1462
Epoch 3/10
11/11 [==============================] - 0s 3ms/step - loss: 147.3134 - mean_absolute_error: 147.3134 - val_loss: 154.7688 - val_mean_absolute_error: 154.7688
Epoch 4/10
11/11 [==============================] - 0s 4ms/step - loss: 143.9966 - mean_absolute_error: 143.9966 - val_loss: 149.8965 - val_mean_absolute_error: 149.8965
Epoch 5/10
11/11 [==============================] - 0s 4ms/step - loss: 137.2436 - mean_absolute_error: 137.2436 - val_loss: 140.0717 - val_mean_abs

Proviamo ad esportare il miglior modello ottenuto tramite il metodo `export_model()`.

In [40]:
model = reg.export_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10)]              0         
                                                                 
 multi_category_encoding (Mu  (None, 10)               0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 10)               21        
 n)                                                              
                                                                 
 dense (Dense)               (None, 512)               5632      
                                                                 
 re_lu (ReLU)                (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 32)                16416 